### Importing Required Libraries

We start by importing the necessary Python libraries for numerical computation, data visualization, and data handling:

- **NumPy** – for numerical operations  
- **Matplotlib** – for creating visualizations  
- **Pandas** – for data manipulation and analysis  

We also define the mathematical constant **Euler's Number (E)** for later use.


In [ ]:
import numpy as np
# import nnfs
# from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt
import pandas as pd
# nnfs.init()
E = 2.71828182846



-Creating a Dense Layer class for a neural network.
-Initializes small random weights and zero biases.
-Forward pass computes outputs, backward pass calculates gradients for weights, biases, and inputs.

In [ ]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = np.dot(dvalues, self.weights.T)
